In [6]:
import pandas as pd

df=pd.read_excel('/workspaces/vikrant_dubai/RAW/AD/AD 1.xlsx')


,DATE/TIME,SO2,DATA CAP. (SO2),SO2 AVG,NO2,DATA CAP. (NO2),NO2 AVG,O3,DATA CAP. (O3),O3 8H AVG.,...,H2S,TOLUENE,O_XYLENE,ETHYLBENZENE,MP_XYLENE_,BENZENE,CH4,NMHC,THC,NOISE
0,01/01/2023 01:00:00,7.22,NaN,6.5,83.51,NaN,74.33,NaN,NaN,NaN,...,NaN,10.84,1.9,2.03,6.97,5.25,NaN,NaN,NaN,68.13
1,01/01/2023 02:00:00,8.78,NaN,6.5,85.72,NaN,74.33,NaN,NaN,NaN,...,NaN,12.17,2.08,2.43,7.83,6.11,NaN,NaN,NaN,69.77
2,01/01/2023 03:00:00,7.59,NaN,6.5,84.89,NaN,74.33,NaN,NaN,NaN,...,NaN,9.73,1.83,1.93,6.51,4.6,NaN,NaN,NaN,69.39
3,01/01/2023 04:00:00,4.64,NaN,6.5,80.41,NaN,74.33,NaN,NaN,NaN,...,NaN,5.99,1.1,1.27,4.19,3.15,NaN,NaN,NaN,68.96
4,01/01/2023 05:00:00,4.87,NaN,6.5,73.84,NaN,74.33,NaN,NaN,NaN,...,NaN,3.83,0.79,0.98,3.09,1.93,NaN,NaN,NaN,66.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,01/31/2023 18:59:59,NaN,NaN,8.02,NaN,NaN,72,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
739,01/31/2023 19:59:59,9.9,NaN,8.02,114.8,NaN,72,NaN,NaN,NaN,...,NaN,16.32,2.82,4.14,10.84,6.57,NaN,NaN,NaN,67.63
740,01/31/2023 20:59:59,13.58,NaN,8.02,112.77,NaN,72,NaN,NaN,NaN,...,NaN,22.24,5.05,8.18,18.9,8.7,NaN,NaN,NaN,68.32
741,01/31/2023 21:59:59,7.24,NaN,8.02,120.97,NaN,72,NaN,NaN,NaN,...,NaN,24,5.6,8.51,20.38,9.36,NaN,NaN,NaN,69.15


In [8]:
not_present_elements = [elem for elem in L2 if elem not in df.columns.tolist()]
not_present_elements

[]